# 作業

以正確的副檔名下載網頁中的圖片
範例網頁:https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html

Note：因為 PTT 會詢問「是否滿 18 歲」，這邊可以用 cookies 繞過

`requests.get(URL, cookies={'over18': '1'}`


In [5]:
import requests
import os
import time

from bs4 import BeautifulSoup
from PIL import Image

In [6]:
url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)

In [8]:
# 決定要儲存的資料夾
output_dir = 'downloads'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}
# 定位所有圖片的 tag
image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
for img_tag in image_tags:
    if 'imgur' not in img_tag['href']:
        continue
    img_id = img_tag['href'].split('/')[-1].split('.')[0]
    img_url = f'https://i.imgur.com/{img_id}.jpg'

    time.sleep(5)  # 加入延遲（可以從 2 秒提高到 5 秒）
    
    try:
        with requests.get(img_url, headers=headers, stream=True) as r:
            if r.status_code == 429:
                print("Too many requests. Waiting before retrying...")
                time.sleep(10)  # 等待更長時間再嘗試
                continue
            r.raise_for_status()
            img = Image.open(r.raw)
            img_savename = f'{output_dir}/{img_id}.{img.format.lower()}'
            img.save(img_savename)
            print(f'Save image {img_savename}')
    except requests.exceptions.HTTPError as e:
        print(f"Error fetching {img_url}: {e}")

Save image downloads/Cgb5oo1.jpeg
Save image downloads/MgjHY4f.jpeg
Save image downloads/yeoRtAs.jpeg
Save image downloads/wsauNEW.jpeg
Save image downloads/IjiETcs.jpeg
